In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from Agents.DQNAgent import DQNAgent
from Boards.Speed_leedo_2p import FullBoard
import sys
import tqdm
import warnings
import os
import pickle
from multiprocessing import Pool
from tornado import concurrent
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

env = FullBoard()

# 0 - load data, 1 - read/process, 2 - do nothing
main_data = 2
game_history_data = 2
state_process = 0
data_q_val = 1

post = 'sep_11_to_sep_15'

2022-09-20 17:25:12.432217: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
if main_data != 2:
    if main_data:
        query="""
            select *
            from `analytics-156605.barath.ludo_details_1`
        """

        df = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

        df.to_feather('../data/ludo_2p_data_' + post)

    else:
        
        df = pd.read_feather('../data/ludo_2p_data_' + post)

    df = df.to_dict('records')

Downloading:   0%|          | 0/462626 [00:00<?, ?rows/s]

In [3]:
# if game_history_data != 2:
#     if game_history_data:

#         query="""
#                 select *
#                 from `analytics-156605.barath.game_history`
#             """

#         game_history = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

#         game_history.to_feather('../data/game_history_1_to_15.feather')
        
#     else:

#         game_history = pd.read_feather('../data/game_history_1_to_15.feather')

#     game_history = game_history.to_dict('records')
#     game_history = {item['user_id']:item['f0_'] for item in game_history if item is not None}
#     # game_history = game_history.groupby('user_id').apply(lambda x: dict(zip(x.game_ref_id, x.last_5))).to_dict()

In [4]:
if state_process != 2:
    if state_process:

        warnings.filterwarnings('ignore')
        
        def func(group):

            def extract_pos(x):
                return np.array([json.loads(i) for i in x])

            def extract_home(x):
                temp = np.array([json.loads(i) for i in x])
                temp[temp=='HOME'] = 1
                temp[temp!='HOME'] = 0
                temp = temp.astype('float32')
                temp = np.sum(temp, -1)
                return np.expand_dims(temp, -1)

            def my_argmax(a):
                rows = np.where(a == a.max(axis=1)[:, None])[0]
                rows_multiple_max = rows[:-1][rows[:-1] == rows[1:]]
                my_argmax = a.argmax(axis=1)
                my_argmax[rows_multiple_max] = -1
                return my_argmax

            try:

                if len(group['pawns_position_1']) > 25 and len(group['pawns_position_2']) > 25:
                    
                    pawn_position_1 = extract_pos(group['pawns_position_1'])
                    pawn_position_2 = extract_pos(group['pawns_position_2'])
                    home_1 = extract_home(group['home_1'])
                    home_2 = extract_home(group['home_2'])

                    if len(pawn_position_1) != len(pawn_position_2):
                        length = np.min([len(pawn_position_1), len(pawn_position_2)])
                        pawn_position_1 = pawn_position_1[:length]
                        pawn_position_2 = pawn_position_2[:length]
                        home_1 = home_1[:length]
                        home_2 = home_2[:length]

                    first_user = group['from_user'][0]
                    second_user = group['opponent'][0]
                    
                    from_user = np.array(group['from_user'])
                    dice = np.array(group['dice_score'])

                    user_ids = (from_user == second_user).astype('float32')
                    user_ids = user_ids[1:]
                    dice = dice[1:]

                    pos = np.concatenate((pawn_position_1, pawn_position_2), axis=-1)
                    home = np.concatenate((home_1, home_2), -1)
                    score = [np.sum(json.loads(group['score_1'][-1])), np.sum(json.loads(group['score_2'][-1]))]
                    mu_0 = group['mu'][0]
                    sigma_0 = group['sigma'][0]
                    idx = np.where(group['from_user'] == second_user)[0][0]
                    mu_1 = group['mu'][idx]
                    sigma_1 = group['sigma'][idx]

                    score = np.array(score)

                    action_1 = my_argmax((pawn_position_1[1:] - pawn_position_1[:-1]))
                    action_2 = my_argmax((pawn_position_2[1:] - pawn_position_2[:-1]))
                    action = np.copy(action_1)
                    action[action==-1] = action_2[action==-1]

                    states = np.concatenate((np.expand_dims(dice, -1).astype('float32'), pos[:-1,:], home[:-1,:]), -1)

                    return {'division': group['division'], 'user_id_0': first_user, 'user_id_1': second_user,
                                'user_ids': user_ids, 'score': score, 
                                'states': states, 'action': action,
                                'mu_0': mu_0, 'sigma_0': sigma_0,
                                'mu_1': mu_1, 'sigma_1': sigma_1}
            except:
                return None

        p = Pool(16)

        results = p.map(func, tqdm.tqdm(df))

        results = [i for i in results if i is not None]
        
        with open('../data/state_data_' + post + '.pkl', 'wb') as f:
            pickle.dump(results, f)

    else:
        
        with open('../data/state_data_' + post + '.pkl', 'rb') as f:
            results = pickle.load(f)

In [ ]:
if data_q_val:
    
    agent0 = DQNAgent(env.state_size(), env.action_size(), env.max_val())
    agent0.load('./model_output/DQN_2p_v2/0004/weights_100000.hdf5')

    # for dat in tqdm.tqdm(data):
        
        # group = data[dat]

    def func1(group):

        if len(np.squeeze(np.array(group['states'])).shape) == 2:

            q_vals = np.squeeze(agent0.model.predict(np.reshape(group['states'], [len(group['states']),11,1])/69))
            q_temp = np.array([q_vals[i,group['action'][i]] for i in range(len(group['states']))])
            q_max = np.max(q_vals, -1)
            q_min = np.min(q_vals, -1)
            # data[dat].update({'q_user_0': q_temp[group['user_ids']==0],
            #                   'q_user_1': q_temp[group['user_ids']==1],
            #                   'q_max_user_0': q_max[group['user_ids']==0],
            #                   'q_max_user_1': q_max[group['user_ids']==1]})
            
            return {'q_user_0': q_temp[group['user_ids']==0],
                    'q_user_1': q_temp[group['user_ids']==1],
                    'q_max_user_0': q_max[group['user_ids']==0],
                    'q_max_user_1': q_max[group['user_ids']==1],
                    'q_min_user_0': q_min[group['user_ids']==0],
                    'q_min_user_1': q_min[group['user_ids']==1]}

    # futures = []
    # with concurrent.futures.ThreadPoolExecutor(max_workers=24) as executor:
    #     for d in tqdm.tqdm(results):
    #         future = executor.submit(func1, d)
    #         futures.append(future)

    results1 = []
    for i, d in enumerate(tqdm.tqdm(results)):
        results1.append(func1(d))

        if i % 100000 == 0:
            with open('../data/q_val_data_' + post + '.pkl', 'wb') as f:
                pickle.dump([results, results1], f)

    # for future in tqdm.tqdm(futures):
    #     results1.append(future.result())

    with open('../data/q_val_data_' + post + '.pkl', 'wb') as f:
        pickle.dump([results, results1], f)

else:

    with open('../data/q_val_data_' + post + '.pkl', 'rb') as f:
        results, results1 = pickle.load(f)